# Aula 8 - Solução dos exercícios - Parte 2 - Geração dos exemplos negativos

Leandro Carísio Fernandes

<br>



In [ ]:
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

!pip install pyserini -q
!pip install faiss-cpu -q
!pip install wget -q
!pip install -qU huggingface_hub datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%time
from pathlib import Path
import wget
import json
import zipfile
    
if not Path('./collections/trec-covid.zip').is_file():
    !mkdir collections
    wget.download(url_trec_covid, out='./collections/')
    with zipfile.ZipFile('./collections/trec-covid.zip', 'r') as zip_ref:
        zip_ref.extractall('./collections/')
    
def carrega_corpus_trec_covid():
    retorno = {}
    with open('./collections/trec-covid/corpus.jsonl') as corpus:
        for i, line in enumerate(corpus):
            doc = json.loads(line)
            retorno[doc['_id']] = doc
            if (i % 10000 == 0):
                print(f'Processado {i} documentos')
    return retorno

corpus_trec_covid = carrega_corpus_trec_covid()

Processado 0 documentos
Processado 10000 documentos
Processado 20000 documentos
Processado 30000 documentos
Processado 40000 documentos
Processado 50000 documentos
Processado 60000 documentos
Processado 70000 documentos
Processado 80000 documentos
Processado 90000 documentos
Processado 100000 documentos
Processado 110000 documentos
Processado 120000 documentos
Processado 130000 documentos
Processado 140000 documentos
Processado 150000 documentos
Processado 160000 documentos
Processado 170000 documentos
CPU times: user 3.85 s, sys: 1.01 s, total: 4.86 s
Wall time: 10.7 s


Recupera classes do BM25 de exercícios anteriores:

In [ ]:
%%time
from pyserini.analysis import Analyzer, get_lucene_analyzer
from collections import Counter
import array
import pickle
import math

# Definição de uma classe para índice invertido
class IndiceInvertido:
  lista_stopwords_nltk = set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.lucene_analyser = Analyzer(get_lucene_analyzer(stemmer='porter'))

  def tokenizar(self, texto):
    return self.lucene_analyser.analyze(texto)

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)
    
    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)
  
  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])   
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      
      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 2.08 ms, sys: 0 ns, total: 2.08 ms
Wall time: 5.36 ms


In [ ]:
iidx_trec_covid = IndiceInvertido()

# Carrega todo o corpus no índice invertido
cnt = 0
for id, doc in corpus_trec_covid.items():
  # O foco do índice aqui é amostrar documentos "negativos". Então vamos indexar apenas
  # os documentos que tem texto, pra não ter risco de ficar retornando documento vazio depois
  if doc['text'] != '':
    iidx_trec_covid.adiciona_doc(id, f"{doc['title']} {doc['text']}")

  cnt += 1
  if cnt % 10000 == 0:
    print(f'{cnt} docs indexados%')


10000 docs indexados%
20000 docs indexados%
30000 docs indexados%
40000 docs indexados%
50000 docs indexados%
60000 docs indexados%
70000 docs indexados%
80000 docs indexados%
90000 docs indexados%
100000 docs indexados%
110000 docs indexados%
120000 docs indexados%
130000 docs indexados%
140000 docs indexados%
150000 docs indexados%
160000 docs indexados%
170000 docs indexados%


Lê a base salva no repositório:

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import datasets
ds = datasets.load_dataset('unicamp-dl/trec-covid-experiment')

import pandas as pd
df = pd.concat((v.to_pandas().assign(origin=k) for k,v in ds.items()),
               ignore_index=True)
df

  0%|          | 0/4 [00:00<?, ?it/s]

,query,positive_doc_id,negative_doc_ids,origin
0,This is a example query 1,doc1,"[xxx, yyy, zzz]",example
1,This is another example query,doc2,"[aaa, bbb, ccc]",example
2,Example of query with no negative doc_ids,doc2,[],example
3,This is a example query 1 (file 2),doc12222,"[xxx, yyy, zzz]",example2
4,This is another example query (file 2),doc12345,"[aaa, bbb, ccc]",example2
...,...,...,...,...
1465,What are the recent nanoscience advancements f...,8nwwhu9d,"[05u5q8ya, lkrlt4pa, ape10tbe, fw6dorow, gwoy8...",leandro_carisio_01
1466,What is the role of neutrophils in the pathoge...,c4wy70hf,"[co2fgz26, dotjy9ur, va7nmsur, x16659y5, gqjcv...",leandro_carisio_01
1467,Why is pulmonary metastasectomy widely and inc...,xv47an66,"[0i61q99c, 3ppofm2k, fgjzo4xq, krgyafvf, dhd8o...",leandro_carisio_01
1468,How does the use of concurrent suction while d...,bjozlk01,"[34qsj74a, d9bwlx4a, ya9klbbm, qxy2xb5h, 6ic2f...",leandro_carisio_01


In [ ]:
pd.unique(df.origin)

array(['example', 'example2',
       'eduseiti_100_queries_expansion_20230428_01', 'leandro_carisio_01'],
      dtype=object)

In [ ]:
dados_pra_colocar_neg_id = df[df.origin == 'leandro_carisio_01']

In [ ]:
import random

n_resultados_bm25 = 1000
n_negative_doc_ids = 5
buscador = BM25(iidx_trec_covid, 0.82, 0.68, 1)

arquivo_saida = 'leandro_carisio_01.jsonl'

cnt = 0
with open(arquivo_saida, "w") as f:
  for index, row in dados_pra_colocar_neg_id.iterrows():
    query = row['query']

    doc_score = buscador.pesquisar(query)
    docs, _ = zip(*doc_score)
    docs = docs[0:(min(n_resultados_bm25, len(doc_score)))]
    negative_doc_ids = random.sample(docs, n_negative_doc_ids)

    # Só se precisar dos dados no dataframe
    row['negative_doc_ids'] = negative_doc_ids

    linha_jsonl = {
        "positive_doc_id": row["positive_doc_id"],
        "query": query,
        "negative_doc_ids": negative_doc_ids
    }
    json.dump(linha_jsonl, f)
    f.write("\n")

    cnt += 1
    if (cnt % 100 == 0):
      print(f'{cnt} negative_doc_ids processados')

100 negative_doc_ids processados
200 negative_doc_ids processados
300 negative_doc_ids processados
400 negative_doc_ids processados
500 negative_doc_ids processados
600 negative_doc_ids processados
700 negative_doc_ids processados
800 negative_doc_ids processados
900 negative_doc_ids processados
1000 negative_doc_ids processados


In [ ]:
# Depois de subir pro repositório, testa se está correto:

import datasets
ds = datasets.load_dataset('unicamp-dl/trec-covid-experiment')

import pandas as pd
df = pd.concat((v.to_pandas().assign(origin=k) for k,v in ds.items()),
               ignore_index=True)
df

  0%|          | 0/4 [00:00<?, ?it/s]

,query,positive_doc_id,negative_doc_ids,origin
0,This is a example query 1,doc1,"[xxx, yyy, zzz]",example
1,This is another example query,doc2,"[aaa, bbb, ccc]",example
2,Example of query with no negative doc_ids,doc2,[],example
3,This is a example query 1 (file 2),doc12222,"[xxx, yyy, zzz]",example2
4,This is another example query (file 2),doc12345,"[aaa, bbb, ccc]",example2
...,...,...,...,...
1465,What are the recent nanoscience advancements f...,8nwwhu9d,"[05u5q8ya, lkrlt4pa, ape10tbe, fw6dorow, gwoy8...",leandro_carisio_01
1466,What is the role of neutrophils in the pathoge...,c4wy70hf,"[co2fgz26, dotjy9ur, va7nmsur, x16659y5, gqjcv...",leandro_carisio_01
1467,Why is pulmonary metastasectomy widely and inc...,xv47an66,"[0i61q99c, 3ppofm2k, fgjzo4xq, krgyafvf, dhd8o...",leandro_carisio_01
1468,How does the use of concurrent suction while d...,bjozlk01,"[34qsj74a, d9bwlx4a, ya9klbbm, qxy2xb5h, 6ic2f...",leandro_carisio_01


In [ ]:
# Query: What is SARS-CoV-2 and how is it transmitted?
# Exemplo positivo:
print(corpus_trec_covid['0iq9s94n'])
print('***********')
# Exemplo negativo:
print(corpus_trec_covid['1yf1ae1d'])
print(corpus_trec_covid['bxw5pnjy'])
print(corpus_trec_covid['eq2ahtlc'])
print(corpus_trec_covid['fk0ri80v'])

{'_id': '0iq9s94n', 'title': 'A Guide to COVID‐19: a global pandemic caused by the novel coronavirus SARS‐CoV‐2', 'text': 'The emergence of the SARS‐CoV‐2 strain of the human coronavirus has thrown the world into the midst of a new pandemic. In the human body, the virus causes COVID‐19, a disease characterized by shortness of breath, fever, and pneumonia, which can be fatal in vulnerable individuals. SARS‐CoV‐2 has characteristics of past human coronaviruses, with close genomic similarities to SARS‐CoV, the virus that causes the disease SARS. Like these related coronaviruses, SARS‐CoV‐2 is transmitted through the inhalation of droplets and interaction with contaminated surfaces. Across the world, laboratories are developing candidate vaccines for the virus – with vaccine trials underway in the US and the United Kingdom ‐ and considering various drugs for possible treatments and prophylaxis. Here, we provide an overview of SARS‐CoV‐2 by analyzing its virology, epidemiology, and modes of